In [1]:
import eospy
from eospy.cleos import Cleos
import pytz
import random
import datetime
import time
import csv
import numpy as np

import pandas as pd

import json

ce = Cleos(url='http://140.113.107.97:8800')
info = ce.get_info()

In [2]:
print(info)

{'server_version': 'de78b49b', 'chain_id': 'b7d777c05f49ad3cfad432b75434a7119c353f9788335b9b47772687542fef6d', 'head_block_num': 7210, 'last_irreversible_block_num': 6880, 'last_irreversible_block_id': '00001ae0b9420581db0c814ac71b6b133a6faad5c6acbaff18cdbb5b1ebf5a36', 'head_block_id': '00001c2a6ff233818aeba8a8c4019b233bbc004fb303b1f0920c1618d52a030a', 'head_block_time': '2020-06-21T08:21:38.500', 'head_block_producer': 'userqsylizkn', 'virtual_block_cpu_limit': 200000000, 'virtual_block_net_limit': 1048576000, 'block_cpu_limit': 199900, 'block_net_limit': 1048576, 'server_version_string': 'v2.0.5', 'fork_db_head_block_num': 7210, 'fork_db_head_block_id': '00001c2a6ff233818aeba8a8c4019b233bbc004fb303b1f0920c1618d52a030a', 'server_full_version_string': 'v2.0.5-de78b49b5765c88f4e005046d1489c3905985b94'}


In [3]:
data_index = ['txid', 'customer', 'temp_cus', 'time_c', 'r_w', 'temp_rw', 'time_rw', 'r_w', 'temp_rw', 'time_rw', 'de', 'temp_de', 'time_de']

In [4]:
format(datetime.datetime.now())[:-7]

'2020-06-21 16:21:38'

In [5]:
data_save = []

In [6]:
class block_pusher():
    def __init__(self, endpoint, contract_account):
        self.ce = Cleos(url=endpoint)
        self.contract_account = contract_account
        
        self.user_list = []
        self.restaurant_list = []
        self.deliver_man_list = []
        
        self.tasks = []
        
    def add_user(self, account_name, account_active_private_key):
        self.user_list.append([account_name, account_active_private_key])
        
    def add_restaurant(self, account_name, account_active_private_key):
        self.restaurant_list.append([account_name, account_active_private_key])
        
    def add_deliver_man(self, account_name, account_active_private_key):
        self.deliver_man_list.append([account_name, account_active_private_key])
        
    def create_tx_id(self):
#         id_map = '12345abcdefghijklmnopqrstuvwxyz'
#         id_tx = ''
#         for i in range(10):
#             sele = random.randint(0,len(id_map)-1)
#             id_tx = id_tx + id_map[sele]
#         return id_tx
        return format(random.randint(10000000,99999999))
        
    def create_tasks(self, mount_out_task = 50):
        self.tasks = []
        for i in range(mount_out_task):
            
            tx_id = self.create_tx_id()
            
            user = random.randint(0,len(pub.user_list)-1)
            restaurant = random.randint(0,len(pub.restaurant_list)-1)
            deliver_man = random.randint(0,len(pub.deliver_man_list)-1)
            
            user_temp = 36+random.randint(0,15)/10
            cook_temp = 36+random.randint(0,15)/10
            deliver_temp = 36+random.randint(0,15)/10
            
            self.tasks.append([tx_id, user, restaurant, deliver_man, user_temp, cook_temp, deliver_temp])
            
    def push_contract(self, action , tx_id, account, account_key, temperture):
        ##################################################################################################################
        if (action in 'c2r'):
            arguments = {
                "txid": tx_id,  # sender
                "customer": account,  # receiver
                "temp_cus": format(temperture),  # In EOS
                "time_c": format(datetime.datetime.now())[:-7],
            }
        elif (action in 'rc2rw'):
            arguments = {
                "txid": tx_id,  # sender
                "r_cook": account,  # receiver
                "temp_rc": format(temperture),  # In EOS
                "time_rc": format(datetime.datetime.now())[:-7],
            }
        elif (action in 'rw2de'):
            arguments = {
                "txid": tx_id,  # sender
                "r_w": account,  # receiver
                "temp_rw": format(temperture),  # In EOS
                "time_rw": format(datetime.datetime.now())[:-7],
            }
        elif (action in 'de2c'):
            arguments = {
                "txid": tx_id,  # sender
                "de": account,  # receiver
                "temp_de": format(temperture),  # In EOS
                "time_de": format(datetime.datetime.now())[:-7],
            }
        ##################################################################################################################
        print("arg = ",arguments)
        payload = {
            "account": self.contract_account,
            "name": action,
            "authorization": [{
                "actor": account,
                "permission": "active",
            }],
        }
        print("pay = ",payload)
        #print("key = ",account_key)
        #Converting payload to binary
        data = ce.abi_json_to_bin(payload['account'], payload['name'], arguments)
        #Inserting payload binary form as "data" field in original payload
        payload['data'] = data['binargs']
        #final transaction formed
        trx = {"actions": [payload]}
        import datetime as dt
        trx['expiration'] = str(
            (dt.datetime.utcnow() + dt.timedelta(seconds=60)).replace(tzinfo=pytz.UTC))
        # use a string or EOSKey for push_transaction
        # key = "5KQwrPbwdL6PhXujxW37FSSQZ1JiwsST4cqQzDeyXtP79zkvFD3"
        # use EOSKey:
        key = eospy.keys.EOSKey(account_key)
        resp = ce.push_transaction(trx, key, broadcast=True)
        print('------------------------------------------------')
        #print(resp)
        #print('------------------------------------------------')
        
    def push_c2r(self, tx_id, index, user_temp):
        self.push_contract('c2r', tx_id, self.user_list[index][0], self.user_list[index][1],user_temp)
        
    def push_rc2rw(self, tx_id, index, restaurant_temp):
        self.push_contract('rc2rw', tx_id, self.restaurant_list[index][0], self.restaurant_list[index][1], restaurant_temp)
        
    def push_rw2de(self, tx_id, index, restaurant_temp):
        # a little be different, this also push restaurant's account
        self.push_contract('rw2de', tx_id, self.restaurant_list[index][0], self.restaurant_list[index][1], restaurant_temp)
    
    def push_de2c(self, tx_id, index, deliver_man_temp):
        # a little be different, this push  delivery mam's account
        self.push_contract('de2c', tx_id, self.deliver_man_list[index][0], self.deliver_man_list[index][1], deliver_man_temp)

    def run_tasks(self):
        #    0.     1.      2.          3.           4.        5.            6. 
        # [tx_id, user, restaurant, deliver_man, user_temp, cook_temp, deliver_temp]
        
        for i in range(len(self.tasks)):
            test_time = 0
#             print("push de2c")
#             self.push_de2c(self.tasks[i][0], self.tasks[i][3], self.tasks[i][4])
            while True:
                try:
                    test_time += 1
                    print("push cr2")
                    self.push_c2r(self.tasks[i][0], self.tasks[i][1], self.tasks[i][4])
                    time.sleep(2)

                    print("push rc2rw")
                    self.push_rc2rw(self.tasks[i][0], self.tasks[i][2], self.tasks[i][5])
                    time.sleep(2)

                    print("push rw2de")
                    self.push_rw2de(self.tasks[i][0], self.tasks[i][2], self.tasks[i][5])
                    time.sleep(2)

                    print("push de2c")
                    self.push_de2c(self.tasks[i][0], self.tasks[i][3], self.tasks[i][6])
                    time.sleep(2)
                except:
                    if(test_time<5):
                        print(">>AGAIN<<")
                        continue
                    print("ABORT")
                break
            print("[TASK : ",i+1,"/",len(self.tasks)," done]")

        

In [7]:
# pub = block_pusher('http://140.113.107.97:8800', 'ozjkloygsxzk')

In [8]:
# with open('/root/notebooks/final/eos_keys.txt', newline='') as f:
#     reader = csv.reader(f)
#     data = list(reader)

In [9]:

# pub.add_user(data[0][0],data[0][1])
# pub.add_user(data[1][0],data[1][1])
# pub.add_user(data[2][0],data[2][1])
# pub.add_user(data[3][0],data[3][1])
# pub.add_user(data[4][0],data[4][1])
# pub.add_user(data[5][0],data[5][1])
# pub.add_user(data[6][0],data[6][1])
# pub.add_user(data[7][0],data[7][1])

# # pub.add_user("123","123")
# # pub.add_user("123","123")

# pub.add_restaurant(data[8][0],data[8][1])
# pub.add_restaurant(data[9][0],data[9][1])
# pub.add_restaurant(data[10][0],data[10][1])
# pub.add_restaurant(data[11][0],data[11][1])
# # pub.add_restaurant("123","123")
# # pub.add_restaurant("123","123")

# pub.add_deliver_man(data[12][0],data[12][1])
# pub.add_deliver_man(data[13][0],data[13][1])
# pub.add_deliver_man(data[14][0],data[14][1])
# pub.add_deliver_man(data[15][0],data[15][1])
# pub.add_deliver_man(data[16][0],data[16][1])
# pub.add_deliver_man(data[17][0],data[17][1])
# pub.add_deliver_man(data[18][0],data[18][1])
# pub.add_deliver_man(data[19][0],data[19][1])
# pub.add_deliver_man(data[20][0],data[20][1])
# pub.add_deliver_man(data[21][0],data[21][1])
# pub.add_deliver_man(data[22][0],data[22][1])
# pub.add_deliver_man(data[23][0],data[23][1])
# # pub.add_deliver_man("123","123")
# # pub.add_deliver_man("123","123")


In [10]:
!wget https://github.com/Intelligent-Systems-Lab/eos-keygen/raw/master/python/account_200.txt
!pwd

--2020-06-21 16:21:39--  https://github.com/Intelligent-Systems-Lab/eos-keygen/raw/master/python/account_200.txt
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/Intelligent-Systems-Lab/eos-keygen/master/python/account_200.txt [following]
--2020-06-21 16:21:40--  https://raw.githubusercontent.com/Intelligent-Systems-Lab/eos-keygen/master/python/account_200.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.76.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.76.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 31800 (31K) [text/plain]
Saving to: ‘account_200.txt.6’

account_200.txt.6   100%[===================>]  31.05K  --.-KB/s    in 0.03s   

2020-06-21 16:21:40 (1.03 MB/s) - ‘account_200.txt.6’ saved [31800/31800]

/root/notebooks/fina

In [11]:
user_data = []

input_file = open ('/root/notebooks/final/BlockPanda/python/account_200.txt')
while True:
    data = []
    
    r =  input_file.readline()
    if r not in '':
        data.append(r.split('\n')[0].split('=')[0][:8])
        data.append(r.split('\n')[0].split('=')[1])
    else:
        break

    for i in range(2):
        r =  input_file.readline()
        if r not in '':
            data.append(r.split('\n')[0].split('=')[1])
        else:
            break
    user_data.append(data)


In [12]:
# user0036 for contract deploy
user0036 = user_data[36]

user_data = user_data[37:100]

In [13]:
user_data[:3]

[['user0037',
  'userohiyrxy5',
  '5K4fAxEkrFyYAcwedmAtBXuz6ydXTTBeSApD64h576uqfRQnjkU',
  'EOS5Z8QoH15hKovMWviM4LjEWYvHPaHTF21th5ezNwwNoKRCm5BKt'],
 ['user0038',
  'userzyz43a3j',
  '5KGCSpmVzgtrqZfjWoPHpQSeDXYGM4f9BqKCRrg2oLAz44uTGzP',
  'EOS7w7A7SRUjpvRbhVZimmxQkWm1FR3DsBKbdg7CMUx8R38pqFtGF'],
 ['user0039',
  'user1wxpuahy',
  '5JWcvwETcvHboNSt449rRvcqcPkn88pzwXast4MM7313vyPm84V',
  'EOS5oujsBgCcBYvJZqVmyvCqsh1uBR5qvSeTxUM4f2DET6jcVeX4T']]

In [14]:
print("Total users: ", len(user_data))

Total users:  63


In [15]:
user_data[0][0]

'user0037'

In [16]:
pub = block_pusher('http://140.113.107.97:8800', user0036[1])

In [17]:
for i in range(len(user_data)):
    r  = random.randint(0,100)
    if r<50:
        pub.add_user(user_data[i][1],user_data[i][2])
        
    if r>=50 and r<75:
        pub.add_restaurant(user_data[i][1],user_data[i][2])
    
    if r>=75 and r<=100:
        pub.add_deliver_man(user_data[i][1],user_data[i][2])

In [18]:
pub.create_tasks(30)

In [19]:
pub.tasks

[['39649026', 14, 3, 0, 36.6, 36.2, 36.1],
 ['20223326', 26, 10, 0, 37.2, 37.2, 36.3],
 ['48750327', 6, 6, 12, 37.5, 37.3, 37.5],
 ['89353041', 17, 11, 11, 36.0, 37.3, 37.5],
 ['21307743', 2, 8, 4, 36.2, 36.5, 36.1],
 ['63930946', 13, 13, 14, 36.9, 36.5, 36.6],
 ['51211516', 29, 3, 3, 37.2, 36.1, 36.4],
 ['93134566', 29, 11, 3, 36.2, 37.2, 36.7],
 ['93990470', 22, 0, 11, 36.3, 36.3, 36.7],
 ['97864633', 0, 13, 10, 36.8, 36.9, 36.4],
 ['24310672', 2, 8, 10, 36.0, 36.9, 36.4],
 ['90308881', 32, 4, 3, 36.9, 37.3, 37.4],
 ['96238744', 18, 11, 8, 37.4, 37.4, 36.1],
 ['84249826', 3, 6, 14, 37.5, 37.3, 36.7],
 ['72002113', 7, 13, 14, 36.5, 37.1, 36.4],
 ['97689264', 32, 12, 15, 36.4, 36.8, 37.0],
 ['29124876', 24, 7, 5, 36.2, 37.1, 36.8],
 ['39878986', 6, 6, 1, 36.8, 36.6, 37.3],
 ['11797639', 7, 0, 13, 37.1, 37.5, 36.5],
 ['60564257', 22, 3, 4, 37.3, 36.3, 36.5],
 ['53891641', 31, 10, 6, 36.7, 36.6, 37.5],
 ['44805267', 1, 1, 11, 36.2, 36.3, 36.7],
 ['92678202', 20, 9, 14, 36.7, 37.3, 37.0],

In [20]:
now = format(datetime.datetime.now())[:-7]

In [21]:
pub.run_tasks()

push cr2
arg =  {'txid': '39649026', 'customer': 'userjlh3svzc', 'temp_cus': '36.6', 'time_c': '2020-06-21 16:21:41'}
pay =  {'account': 'user5pdpw5aq', 'name': 'c2r', 'authorization': [{'actor': 'userjlh3svzc', 'permission': 'active'}]}
------------------------------------------------
push rc2rw
arg =  {'txid': '39649026', 'r_cook': 'userilr3qhda', 'temp_rc': '36.2', 'time_rc': '2020-06-21 16:21:43'}
pay =  {'account': 'user5pdpw5aq', 'name': 'rc2rw', 'authorization': [{'actor': 'userilr3qhda', 'permission': 'active'}]}
------------------------------------------------
push rw2de
arg =  {'txid': '39649026', 'r_w': 'userilr3qhda', 'temp_rw': '36.2', 'time_rw': '2020-06-21 16:21:45'}
pay =  {'account': 'user5pdpw5aq', 'name': 'rw2de', 'authorization': [{'actor': 'userilr3qhda', 'permission': 'active'}]}
------------------------------------------------
push de2c
arg =  {'txid': '39649026', 'de': 'userzyz43a3j', 'temp_de': '36.1', 'time_de': '2020-06-21 16:21:47'}
pay =  {'account': 'user5

In [22]:
get_table = ce.get_table(user0036[1],user0036[1], 'people',lower_bound='0',limit=-1)

In [23]:
data_save = []
for i in get_table['rows']:
    data = []
    for k in data_index:
        data.append(i[k])
    data_save.append(data)

In [24]:
df = pd.DataFrame(data_save)
df_new = df.rename(columns=lambda s: data_index[s])

In [25]:
for i in data_index:
    df_new[i] = df_new[i].replace('', np.nan)
df_new = df_new.dropna()

In [26]:
now_ = format(datetime.datetime.now())[:-7]

lower_bound = df_new['time_c'] > now
upper_bound = df_new['time_c'] < now_

print("Time range:")
print("From : ",now)
print("TO   : ",now_)

Time range:
From :  2020-06-21 16:21:41
TO   :  2020-06-21 16:25:59


In [27]:
df_new = df_new.sort_values(by=['time_c'])
df_new = df_new[lower_bound & upper_bound]

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


In [28]:
df_new

,txid,customer,temp_cus,time_c,r_w,temp_rw,time_rw,r_w,temp_rw,time_rw,de,temp_de,time_de
1,20223326,useretpjamyd,37.2,2020-06-21 16:21:50,user1nsm5zfu,37.2,2020-06-21 16:21:54,user1nsm5zfu,37.2,2020-06-21 16:21:54,userzyz43a3j,36.3,2020-06-21 16:21:56
11,48750327,useropuolsdn,37.5,2020-06-21 16:21:58,user4si1ij5d,37.3,2020-06-21 16:22:02,user4si1ij5d,37.3,2020-06-21 16:22:02,usera5g4kxww,37.5,2020-06-21 16:22:05
22,89353041,userghljjogw,36.0,2020-06-21 16:22:07,useruxh2ghjw,37.3,2020-06-21 16:22:11,useruxh2ghjw,37.3,2020-06-21 16:22:11,user1wt4iq5q,37.5,2020-06-21 16:22:13
2,21307743,userfpdst22o,36.2,2020-06-21 16:22:15,userkufhnawp,36.5,2020-06-21 16:22:20,userkufhnawp,36.5,2020-06-21 16:22:20,usertirqca52,36.1,2020-06-21 16:22:22
16,63930946,user4gyrbgr3,36.9,2020-06-21 16:22:24,usernp4dq5ir,36.5,2020-06-21 16:22:28,usernp4dq5ir,36.5,2020-06-21 16:22:28,userpa1h1ovq,36.6,2020-06-21 16:22:30
12,51211516,usernyvyxz11,37.2,2020-06-21 16:22:33,userilr3qhda,36.1,2020-06-21 16:22:37,userilr3qhda,36.1,2020-06-21 16:22:37,uservd2rf2rr,36.4,2020-06-21 16:22:39
26,93134566,usernyvyxz11,36.2,2020-06-21 16:22:41,useruxh2ghjw,37.2,2020-06-21 16:22:45,useruxh2ghjw,37.2,2020-06-21 16:22:45,uservd2rf2rr,36.7,2020-06-21 16:22:48
27,93990470,userpmdua1aq,36.3,2020-06-21 16:22:50,userpmhftjtk,36.3,2020-06-21 16:22:54,userpmhftjtk,36.3,2020-06-21 16:22:54,user1wt4iq5q,36.7,2020-06-21 16:22:56
30,97864633,userohiyrxy5,36.8,2020-06-21 16:22:58,usernp4dq5ir,36.9,2020-06-21 16:23:03,usernp4dq5ir,36.9,2020-06-21 16:23:03,userkjtbilqd,36.4,2020-06-21 16:23:05
3,24310672,userfpdst22o,36.0,2020-06-21 16:23:07,userkufhnawp,36.9,2020-06-21 16:23:11,userkufhnawp,36.9,2020-06-21 16:23:11,userkjtbilqd,36.4,2020-06-21 16:23:13
